# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-05-12 06:54:33] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=30164, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=949680383, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1,

[2025-05-12 06:54:42] Attention backend not set. Use fa3 backend by default.
[2025-05-12 06:54:42] Init torch distributed begin.


[2025-05-12 06:54:43] Init torch distributed ends. mem usage=0.00 GB
[2025-05-12 06:54:43] Load weight begin. avail mem=46.29 GB


[2025-05-12 06:54:44] Using model weights format ['*.safetensors']


[2025-05-12 06:54:45] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.08it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.08it/s]

[2025-05-12 06:54:45] Load weight end. type=Qwen2ForCausalLM, dtype=torch.bfloat16, avail mem=45.22 GB, mem usage=1.06 GB.
[2025-05-12 06:54:45] KV Cache is allocated. #tokens: 20480, K size: 0.12 GB, V size: 0.12 GB
[2025-05-12 06:54:45] Memory pool end. avail mem=44.82 GB


[2025-05-12 06:54:46] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-05-12 06:54:47] INFO:     Started server process [2567081]
[2025-05-12 06:54:47] INFO:     Waiting for application startup.
[2025-05-12 06:54:47] INFO:     Application startup complete.
[2025-05-12 06:54:47] INFO:     Uvicorn running on http://0.0.0.0:30164 (Press CTRL+C to quit)
[2025-05-12 06:54:47] INFO:     127.0.0.1:37084 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-12 06:54:48] INFO:     127.0.0.1:37094 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-12 06:54:48] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:54:49] INFO:     127.0.0.1:37102 - "POST /generate HTTP/1.1" 200 OK
[2025-05-12 06:54:49] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-05-12 06:54:52] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:54:52] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, gen throughput (token/s): 6.61, #queue-req: 0


[2025-05-12 06:54:53] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, gen throughput (token/s): 87.78, #queue-req: 0


[2025-05-12 06:54:53] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, gen throughput (token/s): 86.84, #queue-req: 0
[2025-05-12 06:54:53] INFO:     127.0.0.1:37106 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-05-12 06:54:53] INFO:     127.0.0.1:37112 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-05-12 06:54:53] INFO:     127.0.0.1:37124 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-05-12 06:54:53] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:54:54] INFO:     127.0.0.1:37136 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-05-12 06:54:54] INFO:     127.0.0.1:59374 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-05-12 06:54:54] Cache flushed successfully!
[2025-05-12 06:54:54] INFO:     127.0.0.1:59378 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-05-12 06:54:54] Start update_weights. Load format=auto
[2025-05-12 06:54:54] Update engine weights online from disk begin. avail mem=44.62 GB
[2025-05-12 06:54:54] Using model weights format ['*.safetensors']


[2025-05-12 06:54:55] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.14it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.14it/s]

[2025-05-12 06:54:56] Update weights end.
[2025-05-12 06:54:56] Cache flushed successfully!
[2025-05-12 06:54:56] INFO:     127.0.0.1:59380 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-05-12 06:54:56] Start update_weights. Load format=auto
[2025-05-12 06:54:56] Update engine weights online from disk begin. avail mem=44.62 GB


[2025-05-12 06:54:56] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-05-12 06:54:56] INFO:     127.0.0.1:59392 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-12 06:55:02] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=30629, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=282764328, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, bas

[2025-05-12 06:55:09] Downcasting torch.float32 to torch.float16.


[2025-05-12 06:55:10] Overlap scheduler is disabled for embedding models.


[2025-05-12 06:55:10] Downcasting torch.float32 to torch.float16.


[2025-05-12 06:55:11] Attention backend not set. Use fa3 backend by default.
[2025-05-12 06:55:11] Init torch distributed begin.


...

[2025-05-12 06:55:18] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=60.05 GB, mem usage=18.54 GB.
[2025-05-12 06:55:18] KV Cache is allocated. #tokens: 20480, K size: 0.27 GB, V size: 0.27 GB
[2025-05-12 06:55:18] Memory pool end. avail mem=59.23 GB


[2025-05-12 06:55:19] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-12 06:55:19] INFO:     Started server process [2568888]
[2025-05-12 06:55:19] INFO:     Waiting for application startup.
[2025-05-12 06:55:19] INFO:     Application startup complete.
[2025-05-12 06:55:19] INFO:     Uvicorn running on http://0.0.0.0:30629 (Press CTRL+C to quit)
[2025-05-12 06:55:19] INFO:     127.0.0.1:39178 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-12 06:55:20] INFO:     127.0.0.1:39194 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-12 06:55:20] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-12 06:55:21] INFO:     127.0.0.1:39202 - "POST /encode HTTP/1.1" 200 OK
[2025-05-12 06:55:21] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-05-12 06:55:24] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 06:55:24] INFO:     127.0.0.1:39320 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-05-12 06:55:24] Child process unexpectedly failed with an exit code 9. pid=2569034
[2025-05-12 06:55:24] Child process unexpectedly failed with an exit code 9. pid=2569100


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [14]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-12 06:55:31] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=34550, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=964736540, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, downloa

[2025-05-12 06:55:39] Overlap scheduler is disabled for embedding models.


[2025-05-12 06:55:40] Attention backend not set. Use flashinfer backend by default.
[2025-05-12 06:55:40] Init torch distributed begin.


[2025-05-12 06:55:40] Init torch distributed ends. mem usage=0.00 GB
[2025-05-12 06:55:40] Load weight begin. avail mem=46.30 GB


[2025-05-12 06:55:42] Using model weights format ['*.safetensors']


...

[2025-05-12 06:56:03] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-12 06:56:04] INFO:     Started server process [2571542]
[2025-05-12 06:56:04] INFO:     Waiting for application startup.
[2025-05-12 06:56:04] INFO:     Application startup complete.
[2025-05-12 06:56:04] INFO:     Uvicorn running on http://0.0.0.0:34550 (Press CTRL+C to quit)


[2025-05-12 06:56:04] INFO:     127.0.0.1:42336 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-12 06:56:05] INFO:     127.0.0.1:42344 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-12 06:56:05] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-12 06:56:05,756 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


In [15]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

2025-05-12 06:56:59,509 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-12 06:56:59,517 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-12 06:56:59,538 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-12 06:56:59] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-12 06:56:59] INFO:     127.0.0.1:42354 - "POST /encode HTTP/1.1" 200 OK
[2025-05-12 06:56:59] The server is fired up and ready to roll!
2025-05-12 06:56:59,747 - INFO - flashinfer.jit: Loading JIT ops: cascade
2025-05-12 06:56:59,766 - INFO - flashinfer.jit: Finished loading JIT ops: cascade
[2025-05-12 06:56:59] INFO:     127.0.0.1:42370 - "POST /classify HTTP/1.1" 200 OK


In [16]:
terminate_process(reward_process)

[2025-05-12 06:56:59] Child process unexpectedly failed with an exit code 9. pid=2571996


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [17]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-05-12 06:57:06] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=34425, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=523439414, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='

[2025-05-12 06:57:14] Attention backend not set. Use flashinfer backend by default.
[2025-05-12 06:57:14] Init torch distributed begin.


[2025-05-12 06:57:14] Init torch distributed ends. mem usage=0.00 GB
[2025-05-12 06:57:14] Load weight begin. avail mem=60.09 GB


[2025-05-12 06:57:16] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-05-12 06:58:04] INFO:     Started server process [2575151]
[2025-05-12 06:58:04] INFO:     Waiting for application startup.
[2025-05-12 06:58:04] INFO:     Application startup complete.
[2025-05-12 06:58:04] INFO:     Uvicorn running on http://0.0.0.0:34425 (Press CTRL+C to quit)
[2025-05-12 06:58:05] INFO:     127.0.0.1:54232 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-12 06:58:05] INFO:     127.0.0.1:54234 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-12 06:58:05] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-12 06:58:07,089 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-12 06:58:07,113 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-05-12 06:58:07,120 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-12 06:58:07,140 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-12 06:58:07] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-kb-gpu-45/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


2025-05-12 06:58:07,584 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


In [18]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

import glob

output_file = glob.glob("expert_distribution_*.csv")[0]
with open(output_file, "r") as f:
    print_highlight("\n| Layer ID | Expert ID | Count |")
    print_highlight("|----------|-----------|--------|")
    next(f)
    for i, line in enumerate(f):
        if i < 9:
            layer_id, expert_id, count = line.strip().split(",")
            print_highlight(f"| {layer_id:8} | {expert_id:9} | {count:6} |")

2025-05-12 06:58:22,918 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
[2025-05-12 06:58:22] Resetting expert distribution record...
[2025-05-12 06:58:22] INFO:     127.0.0.1:54248 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-05-12 06:58:22] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-05-12 06:58:23] INFO:     127.0.0.1:54246 - "POST /generate HTTP/1.1" 200 OK
[2025-05-12 06:58:23] The server is fired up and ready to roll!


[2025-05-12 06:58:24] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, gen throughput (token/s): 2.33, #queue-req: 0


[2025-05-12 06:58:25] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, gen throughput (token/s): 37.29, #queue-req: 0


[2025-05-12 06:58:26] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, gen throughput (token/s): 37.46, #queue-req: 0


[2025-05-12 06:58:26] INFO:     127.0.0.1:47938 - "POST /generate HTTP/1.1" 200 OK


[2025-05-12 06:58:26] INFO:     127.0.0.1:50780 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-05-12 06:58:26] Resetting expert distribution record...
[2025-05-12 06:58:26] INFO:     127.0.0.1:50784 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [19]:
terminate_process(expert_record_server_process)